# Global Variables

In [1]:
import pandas as pd
import numpy as np
import warnings
import time
import datetime

warnings.filterwarnings('ignore')

exchange = 'NSE'
symbol = "HDFC"

#today = dt.datetime.now()
today = datetime.datetime(2019, 4, 8, 15, 45, 0)

#configuration for downloading data
days = 2 * 365
years = 5
toDate = today
fromDate = toDate - datetime.timedelta(days = days)
freq = '1min'

mode = "Backtesting" # Backtesting/Live
offline = True
if mode =="Live":
    offline = False

KiteAPIKey = "b2w0sfnr1zr92nxm"
KiteAPISecret = "jtga2mp2e5fn29h8w0pe2kb722g3dh1q"

from enum import Enum
class S(Enum):
    BUY = 1
    SELL = 2
    HOLD = 0

%run "KiteConnect_Library.ipynb"    

# Kite - OAuth Login

In [2]:
from kiteconnect import KiteConnect
import platform
from selenium import webdriver
import re

#logging.basicConfig(level=logging.DEBUG)
if offline != True:
    kite = KiteConnect(api_key=KiteAPIKey)
    
if mode == "Live":
    #kite.login_url()

    if platform.system() == "Windows":
        driver = webdriver.Chrome("./automation/chromedriver.exe")
    else:
        driver = webdriver.Chrome("./automation/chromedriver")

    driver.get(kite.login_url())

In [3]:
access_token = ""
if mode == "Live":
    # Redirect the user to the login url obtained
    # from kite.login_url(), and receive the request_token
    # from the registered redirect url after the login flow.
    # Once you have the request_token, obtain the access_token
    # as follows.
    request_token = re.findall("request_token=(.*)&action",driver.current_url)

    data = kite.generate_session(request_token[0], api_secret=KiteAPISecret)
    access_token = data["access_token"]
    kite.set_access_token(access_token)
    print(access_token)
    driver.close()
elif offline != True:
    access_token = "zvyL9tcnN4ANGJmP3tqR7ty5AFaFynPk"
    kite.set_access_token(access_token)

# Download NSE Stock Master List

In [4]:
if offline != True:
    instruments_df = getInstruments(exchange)
    instruments_df.to_hdf('kite_data/kite_data.h5', key=exchange, mode='w')

instruments_df = pd.read_hdf('kite_data/kite_data.h5', key=exchange, mode='r')

EQSYMBOL = lambda x:instruments_df[instruments_df['instrument_token']==x].index[0]
EQTOKEN = lambda x:instruments_df.loc[x,'instrument_token']

# Filter Stocks - portfolio maker

In [5]:
nifty50 = pd.read_csv("data/ind_nifty50list.csv")
niftynext50 = pd.read_csv("data/ind_niftynext50list.csv")
midcap50 = pd.read_csv("data/ind_niftymidcap50list.csv")

downloadlist = nifty50['Symbol']
industry = niftynext50['Industry'].unique()

In [6]:
portfolio = pd.DataFrame(['HDFC', 'SBIN', 'TCS', 'RIIL', 'BHARTIARTL', 'ADANIPORTS', 'DRREDDY'])
portfolioToken = portfolio[0].apply(EQTOKEN)

# Download Historical Data - Equity

In [7]:
if offline != True:
    raw_data = downloadData(symbol, fromDate, toDate)
    raw_data_day = downloadData(symbol,  toDate - dt.timedelta(days = years * 365),toDate, freq="day")


    raw_data_day.to_hdf('kite_data/kite_data.h5', key=symbol+"day",mode='a')
    raw_data.to_hdf('kite_data/kite_data.h5', key=symbol,mode='a')

### Batch Downloader

In [8]:
def batchDownload(downloadlist):
    for symbol in downloadlist:
        print("Downloading data for: "+symbol)

        raw_data = downloadData(symbol, fromDate, toDate)
        raw_data_day = downloadData(symbol,  toDate - dt.timedelta(days = years * 365),toDate, freq="day")

        raw_data_day.to_hdf('kite_data/kite_data.h5', key=symbol+"day",mode='a')
        raw_data.to_hdf('kite_data/kite_data.h5', key=symbol,mode='a')

## Incremental Download

In [9]:
def incrementalDownload(downloadlist):
    for symbol in downloadlist:
        print("Downloading data for: "+symbol)
        tempData = pd.read_hdf('kite_data/kite_data.h5', key=symbol,mode='r')
        fromDate = tempData.index[-1]
        toDate = datetime.datetime.now()
        raw_data = downloadData(symbol, fromDate, toDate)
        tempData = tempData.append(raw_data)
        tempData.to_hdf('kite_data/kite_data.h5', key=symbol,mode='a')
        
        tempData = pd.read_hdf('kite_data/kite_data.h5', key=symbol+"day",mode='r')
        fromDate = tempData.index[-1]
        toDate = datetime.datetime.now()
        raw_data = downloadData(symbol, fromDate, toDate, freq="day")
        
        tempData.to_hdf('kite_data/kite_data.h5', key=symbol+"day",mode='a')
    
    

### Historical Data storage strategy

* minute level data and day level ohlc data is stored in the hd5
* symbol name is used as a key in the hd5 file system

### Tick Data storage strategy
- Tick data should not be merged with the downloaded historical data
- At the start of the trading session, last 60 candles from the historical data is fetched and stored in the dataframe for holding live data
- Two DataFrames ares created for storing live data: 1 for storing timestamp and LTP, another for OHLC
- OHLC data is created by grouping data first based on stock symbol and then based on timestamp(seconds and miliseconds are ignored)
- Streaming data from all the stocks in portfolio are strored in a single table. During post-procession subset of the master table sliced based on symbol is used
- Streaming data is resampled for minute frequency every minute to convert it to OHLC data which is stored in another dataframe
- Symbol is used as a key for the streaming OHLC dataframe storage

# Load Data from disk

In [19]:
raw_data = pd.read_hdf('kite_data/kite_data.h5', key="SBIN",mode='r')
raw_data_day = pd.read_hdf('kite_data/kite_data.h5', key="SBIN"+"day",mode='r')

In [20]:
raw_data_day.head()

,close,high,low,open,volume
date,,,,,
2014-04-10,202.81,204.00,199.02,199.50,28796539
2014-04-11,199.41,202.97,198.44,201.50,20895559
2014-04-15,196.41,200.47,194.76,199.50,16863519
2014-04-16,195.94,199.47,195.01,196.60,19376259
2014-04-17,201.73,202.20,196.28,196.72,21513799


In [21]:
raw_data.index[-1]

Timestamp('2019-04-08 15:29:00')

# Indicators Initialization

In [22]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

noofcandles = 50

yMin = raw_data.iloc[-1*noofcandles:-1]['low'].min()-10
yMax = raw_data.iloc[-1*noofcandles:-1]['high'].max()
xMin = raw_data.index[-1*noofcandles]
xMax = raw_data.index[-1]

# Algorithm

In [25]:
raw_data_day

,close,high,low,open,volume,bbt,bbm,bbb,macd,macdsignal,macdhist,RSI,aroondown,aroonup,obv
date,,,,,,,,,,,,,,,
2014-04-10,202.81,204.00,199.02,199.50,28796539,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.879654e+07
2014-04-11,199.41,202.97,198.44,201.50,20895559,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.969210e+07
2014-04-15,196.41,200.47,194.76,199.50,16863519,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.655562e+07
2014-04-16,195.94,199.47,195.01,196.60,19376259,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.593188e+07
2014-04-17,201.73,202.20,196.28,196.72,21513799,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.074457e+08
2014-04-21,207.21,207.88,202.20,202.20,18872419,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.263181e+08
2014-04-22,204.85,207.70,204.02,207.40,18364099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.446822e+08
2014-04-23,206.52,207.80,204.09,204.50,23265389,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.679476e+08
2014-04-25,208.53,211.16,204.29,206.00,22655729,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.906033e+08


In [27]:


fig = createPlot()

temp_data = raw_data[-20*noofcandles:-1]
temp_data = candlestick(temp_data,1,True)
temp_data = bbands(temp_data,1, True)

temp_data = macd(temp_data,3,True)
temp_data = rsi(temp_data,4, True)
temp_data = aroon(temp_data,5, True)
obv(temp_data,2, True)
#pivotPoint(raw_data, pos=1, plot=False)
#raw_data = calculateStats(raw_data)
#raw_data = emasma(raw_data)
#raw_data = adx(raw_data)
#raw_data = detectCDPattern(raw_data,strPlot="hammer", plot=True)
#raw_data = stoch(raw_data)

# ====== Tradescript Wrapper =======
# Variables
OPEN = 0
CLOSE = 0
HIGH = 0
LOW = 0
VOLUME = 0
BBT = 0
BBM = 0
BBB = 0
AroonUp = 0
AroonDown = 0

UP = 0
DOWN = 1

# Methods
REF = lambda df, i: df.shift(i)
TREND_UP = lambda a, days: ROC(a,days) >= 0.01
TREND_DOWN = lambda a, days: ROC(a,days) <= -0.01
CROSSOVER = lambda a, b: (REF(a,1)<=REF(b,1)) & (a > b)

class algoTrade:
    
    def __init__(s, price):
        global OPEN, CLOSE, HIGH, LOW, VOLUME, BBT, BBM, BBB, AroonDown, AroonUp
        OPEN = price['open']
        CLOSE = price['close']
        HIGH = price['high']
        LOW = price['low']
        VOLUME = price['volume']
        BBT, BBM, BBB = BBANDS( CLOSE, 20,2,2,1)
        AroonDown, AroonUp = AROON(HIGH, LOW, 25)
    
    # Long Strategies
    def long_bb(self):
        return pd.DataFrame( (REF(CLOSE, 1) < REF(BBB, 1)) & (CLOSE > BBB), columns=["buy"] )
    
    def long_bull_engulf_ema(self):
        return pd.DataFrame( (EMA(CLOSE, 9) < EMA(CLOSE, 21)) & (CDLENGULFING(OPEN, HIGH, LOW, CLOSE) == 100) ,columns =["buy"])
    
    def long_hammer(self):
        return pd.DataFrame(TREND_DOWN(CLOSE, 10) & (CDLHAMMER(OPEN, HIGH, LOW, CLOSE) != 0), columns=["buy"])
    
    def long_ichimoku(self):
        C = ( SMA(HIGH, 9) + SMA(LOW, 9) )/2
        D = ( SMA(HIGH, 26) + SMA(LOW, 26) )/2
        A = (C+D)/2
        B = (SMA(HIGH,52)+SMA(LOW,52))/2
        
        return pd.DataFrame( CROSSOVER(A,B) ,columns=["buy"])
    
    def long_aroon(self):
        return pd.DataFrame((AroonUp > 50) & (AroonDown<50), columns=["buy"])
        
    
    # Short Strategies
    def short_bb(self):
        return pd.DataFrame((REF(CLOSE,1) > REF(BBT,1)) & (CLOSE<BBT), columns=["sell"])
    
    def short_bear_engulf(self):
        return pd.DataFrame(TREND_UP(CLOSE,10) & (CDLENGULFING(OPEN, HIGH, LOW, CLOSE) == -100),columns=["sell"])
    
    def short_hanging_man(self):
        return pd.DataFrame(
            TREND_UP(CLOSE, 10) &
            (CDLHANGINGMAN(OPEN, HIGH, LOW, CLOSE) == -100)
            , columns=["sell"])
    
    def short_bear_kicking(self):
        return pd.DataFrame(CDLKICKING(OPEN, HIGH, LOW, CLOSE) != 0, columns=["sell"])
        
    def short_shooting_star(self):
        return pd.DataFrame( TREND_UP(CLOSE, 5) & CDLSHOOTINGSTAR(OPEN, HIGH, LOW, CLOSE) != 0, columns=["sell"])
    
    def short_ichimoku(self):
        C = ( SMA(HIGH, 9) + SMA(LOW, 9) )/2
        D = ( SMA(HIGH, 26) + SMA(LOW, 26) )/2
        A = (C+D)/2
        B = (SMA(HIGH,52)+SMA(LOW,52))/2
        
        return pd.DataFrame( CROSSOVER(B,A) ,columns=["sell"])
    
    def short_aroon(self):
        return pd.DataFrame((AroonUp < 50) & (AroonDown > 50), columns=["sell"])
    
    def tradeDecision(self):
        buy  = self.long_bull_engulf_ema() | self.long_bb() | self.long_ichimoku()
        #buy = self.long_aroon()
        sell = self.short_bear_engulf() | self.short_bb() | self.short_hanging_man()
        #sell = self.short_aroon()
        
        buy['low'] = LOW
        sell['high'] = HIGH
        buy['close'] = sell['close'] = CLOSE

        buy = buy[buy['buy']]
        sell = sell[sell['sell']]
        return (buy,sell)
        
a = algoTrade(temp_data)
buy, sell = a.tradeDecision()
annotateBuySell(buy, "Buy")
annotateBuySell(sell, "Sell")

plotData()

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]
[ (4,1) x1,y4 ]
[ (5,1) x1,y5 ]



# Order Management

In [ ]:
def placeorder(): 
    
    if False:
        # Place an order
        try:
            order_id = kite.place_order(tradingsymbol="INFY",
                                        exchange=kite.EXCHANGE_NSE,
                                        transaction_type=kite.TRANSACTION_TYPE_BUY,
                                        quantity=1,
                                        order_type=kite.ORDER_TYPE_MARKET,
                                        product=kite.PRODUCT_NRML)

            logging.info("Order placed. ID is: {}".format(order_id))
        except Exception as e:
            logging.info("Order placement failed: {}".format(e.message))


def getOrders():    
    # Fetch all orders
    kite.orders()



# WebSocket: Live Tick Handler

In [24]:
if mode=="Live":
    live_data = raw_data.iloc[-30:-1][['open','high','low','close','volume']]


    import pickle
    f = open('live_data_hdfc_full_4_8_2019.txt', 'wb')

    count = 0
    def ticksHandler(ticks):
        global live_data, f, count
        #pickle.dump(ticks, f)
    

In [25]:
#!python
import logging
import multiprocessing
from kiteconnect import KiteTicker
import copy 

logging.basicConfig(level=logging.CRITICAL)

jobs = []

if mode == "Live":
    # Initialise
    kws = KiteTicker(KiteAPIKey, kite.access_token)

    def on_ticks(ws, ticks):
        # Callback to receive ticks.
        #logging.debug("Ticks: {}".format(ticks))
        #count = count + 1

        #print(ticks)
        ticksHandler(ticks)
        #jobs.append(p)
        #p.start()


    def on_connect(ws, response):
        # Callback on successful connect.
        # Subscribe to a list of instrument_tokens (RELIANCE and ACC here).
        ws.subscribe(portfolio)

        # Set RELIANCE to tick in `full` mode.
        # MODE_LTP, MODE_QUOTE, or MODE_FULL

        ws.set_mode(ws.MODE_FULL, [340481, 2953217, 745473]) 
        #ws.set_mode(ws.MODE_LTP, [225537, 3861249]) 
        #ws.set_mode(ws.MODE_MODE_QUOTE, [2714625,779521]) 

    def on_close(ws, code, reason):
        # On connection close stop the main loop
        # Reconnection will not happen after executing `ws.stop()`
        ws.stop()

    # Assign the callbacks.
    kws.on_ticks = on_ticks
    kws.on_connect = on_connect
    #kws.on_close = on_close

    # Infinite loop on the main thread. Nothing after this will run.
    # You have to use the pre-defined callbacks to manage subscriptions.

#p = multiprocessing.Process(target=kws.connect)
#jobs.append(p)
#p.start()


In [26]:
if mode == "Live":
    kws.connect(threaded=True)

In [27]:
if mode == "Live":
    kws.close()
    f.close()

In [28]:
timeStamp = dt.datetime.now().replace(second=0, microsecond=0)
LiveStream = pd.DataFrame()
def pushTick(tick):
    global LiveStream
    tick_df = pd.DataFrame(tick)
    
    try:
        tick_df.loc[tick_df['timestamp'].isna(), 'timestamp'] = timeStamp
        tick_df = tick_df[['timestamp','instrument_token','last_price','volume']]
        tick_df.instrument_token = tick_df.instrument_token.apply(EQSYMBOL)
        tick_df.columns = ['date','symbol','price','volume']
        tick_df.set_index(['symbol','date'], inplace=True)

        LiveStream = LiveStream.append(tick_df)
    except:
        print("Exception occured")

        
live_data = pd.DataFrame()

def resample(freq="1min"):
    global live_data
    LiveStream2 = LiveStream.groupby(['symbol','date']).agg({'price':['first','max','min','last'], 'volume':['last']})
    LiveStream2.columns = LiveStream2.columns.droplevel()
    LiveStream2.columns = ['open', 'high','low','close', 'volume']

    for index, data in LiveStream2.groupby(level=0):
        #print(index)
        sampled = data.loc[index].resample(freq).agg({'open':{'open':'first'},'high':{'high':'max'},'low':{'low':'min'},'close':{'close':'last'},'volume':{'volume':'last'}})
        sampled.columns = sampled.columns.droplevel()
        
        live_data = live_data.append(sampled)

In [29]:
import pickle
#f2 = open('live_data_hdfc_full_4_8_2019.txt', 'rb')

def loadall(filename):
    with open(filename, "rb") as f:
        for count in np.linspace(1,100,100):
            #print(count)
            try:
                tick = pickle.load(f)
                pushTick(tick)
                #print(tick)
            except EOFError:
                break

loadall('kite_data/live_data_hdfc_full_4_8_2019.dat')
resample()
#ticks_df = pickle.load(f2)

Exception occured
Exception occured
Exception occured
Exception occured


In [30]:
live_data.head()

,open,high,low,close,volume
date,,,,,
2019-04-08 23:02:00,380.90,380.90,380.5,380.50,561724
2019-04-08 23:02:00,358.40,358.50,358.1,358.35,2031431
2019-04-08 23:02:00,2762.10,2762.55,2762.1,2762.10,94897
2019-04-08 10:28:00,2050.35,2050.50,2050.0,2050.35,300126
2019-04-08 10:29:00,2050.35,2050.35,2049.1,2049.60,302492


# Visualization

In [17]:
plotData()